### Step 4: Build Upstream Network
This step only needs to be ran once and takes roughly 30 minutes to run.

In [7]:
from utils import build_network
import numpy as np
from timeit import default_timer as timer
from utils import file_management as f_mng


In [10]:
#Read pickled dataframe of all level 12 basins (without geometry)
pkl_df_path = 'data/basins_lvl12_df.pkl'
all_data=f_mng.read_pkl_df(pkl_df_path)

#drop unneeded columns to reduce size and make easier to navigate contents
all_data = all_data.drop(columns=['DIST_SINK','DIST_MAIN','SORT','COAST','MAIN_BAS'])
all_data.head(3)

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,SUB_AREA,UP_AREA,PFAF_ID,ENDO,ORDER
0,1120000010,0,1120000010,11.0,11.0,111011001000,0,0
1,1120000020,0,1120000020,137.0,416.8,111011002100,0,1
2,1121694330,1120000020,1120000020,135.1,280.0,111011002200,0,1


In [11]:
#set segment_id to 0 if endo sink (we don't want to connect to rest of network), or next down if not endo sink
all_data['temp_seg_id'] = np.where(all_data['ENDO']==2, 0, all_data['NEXT_DOWN'])
seg_ids = all_data[['temp_seg_id','HYBAS_ID']]
seg_ids = seg_ids.rename(columns={'temp_seg_id':'seg_id','HYBAS_ID':'upseg_id'})
#merge data with itself to create up_seg dataframe from next_down (reverse from seg, down_seg to seg_upseg) to use existing code
up_seg_df = pd.merge(all_data, seg_ids, left_on='HYBAS_ID', right_on='seg_id', how='left')
#if no value for seg_id, these are headwaters, fill nan with HyBas_ID and upseg_id = 0 (no upstream basin)
values = {'seg_id': up_seg_df['HYBAS_ID'], 'upseg_id': 0}
up_seg_df = up_seg_df.fillna(value=values)
#Remove unneeded columns
up_seg_df = up_seg_df.drop(columns=['HYBAS_ID','NEXT_DOWN','temp_seg_id'])

In [12]:
up_seg_df.head()

,NEXT_SINK,SUB_AREA,UP_AREA,PFAF_ID,ENDO,ORDER,seg_id,upseg_id
0,1120000010,11.0,11.0,111011001000,0,0,1.120000e+09,0.000000e+00
1,1120000020,137.0,416.8,111011002100,0,1,1.120000e+09,1.121694e+09
2,1120000020,135.1,280.0,111011002200,0,1,1.121694e+09,1.121694e+09
3,1120000020,144.9,144.9,111011002300,0,1,1.121694e+09,0.000000e+00
4,1120000030,186.8,186.9,111011003000,0,0,1.120000e+09,0.000000e+00


In [13]:
start= timer()
traverse_queue = build_network.upstream_setup(up_seg_df)
hdf_name = 'output/hb12_netowrk.h5'
build_network.upstream_build_network(traverse_queue, hdf_name)

end = timer() 
print("Time taken:", end-start) 

Time taken: 1642.1929949320001


In [16]:
#For Level12 this took 1642 seconds, this shows minutes
1642/60

27.366666666666667